In [92]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='light')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


In [93]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [94]:
class PriceDP:
    
    def __init__(self, claim):
        self.yluna_price = 'e49168df-26f3-4972-b8c2-309e34e41072'
        self.luna_price = '571e7540-b3cb-458f-82c3-43aae00feae7'
        self.claim = claim
        
    def load(self):
        yluna_price_df = self.claim(self.yluna_price)
        yluna_price_df.columns = [c.lower() for c in yluna_price_df.columns]
        luna_price_df = self.claim(self.luna_price)
        luna_price_df.columns = [c.lower() for c in luna_price_df.columns]
        luna_price_df = luna_price_df.rename(columns={'price':'luna_price'})
        self.yluna_price_df = yluna_price_df
        self.luna_price_df = luna_price_df

In [95]:
dp = PriceDP(claim)
dp.load()

In [96]:
yluna = dp.yluna_price_df[dp.yluna_price_df.offer_asset=='yLuna']
prism = dp.yluna_price_df[dp.yluna_price_df.offer_asset=='PRISM']

In [97]:
df = yluna.merge(prism, on='day', suffixes=['_yluna','_prism'])
df['yluna_price'] = (1/df.belief_price_prism) / df.belief_price_yluna
prices = df
prices = prices.merge(dp.luna_price_df, on='day')[['day','yluna_price','luna_price']]
prices['day'] = prices['day'].apply(lambda x: x[:-13])
prices.sort_values(by='day')

,day,yluna_price,luna_price
32,2022-02-14,25.257616,52.858438
19,2022-02-15,36.966261,55.691098
6,2022-02-16,38.672993,56.467117
10,2022-02-17,36.661442,54.612771
17,2022-02-18,36.349276,50.745742
2,2022-02-19,36.995210,51.183249
0,2022-02-20,35.435485,49.141487
28,2022-02-21,37.993155,51.468442
15,2022-02-22,38.287377,51.570473
21,2022-02-23,43.832131,57.828715


## Query

In [98]:
import requests
luna_apr = float(
    requests.get('https://api.terra.dev/chart/staking-return/annualized').json(
    )[-1]['value']) * 100
et_query = requests.get(
    'https://api.extraterrestrial.money/v1/api/prices').json()
yluna_price = float(et_query['prices']['yLUNA']['price'])
luna_price = float(et_query['prices']['LUNA']['price'])
yluna_apr = luna_apr * luna_price / yluna_price

In [112]:
df = pd.DataFrame(requests.get('https://api.terra.dev/chart/staking-return/annualized').json())
df['date'] = pd.to_datetime(df['datetime'], unit='ms')
df = df[df['date'] > '2021-07-05 15:00:00']
df['day'] = df.date.apply(str)
df.value = df.value.apply(float).apply(lambda x: x*100)
staking_apr = df
staking_apr = staking_apr.rename(columns={'value':'apr'})
staking_apr['day'] = staking_apr['day'].apply(lambda x: x[:-9])

In [115]:
mars_price_chart = alt.Chart(staking_apr).mark_line(point = True).encode(
    x=alt.X('date:T'),
    y=alt.X('apr:Q'),
    tooltip=[alt.Tooltip('date:T', format='%Y-%m-%d %H:%M'),'value:Q']
).properties(width=800).properties(height=300).configure_view(strokeOpacity=0).interactive()
mars_price_chart

alt.Chart(...)

In [116]:
prices_apr = prices.merge(staking_apr, on='day')

In [118]:
prices_apr['yluna_apr'] = prices_apr.apr * prices_apr.luna_price / prices_apr.yluna_price

In [120]:
prices_apr.sort_values(by='day')

,day,yluna_price,luna_price,datetime,apr,date,yluna_apr
31,2022-02-14,25.257616,52.858438,1644850800000,9.566244,2022-02-14 15:00:00,20.019969
18,2022-02-15,36.966261,55.691098,1644937200000,9.542594,2022-02-15 15:00:00,14.376286
5,2022-02-16,38.672993,56.467117,1645023600000,9.269486,2022-02-16 15:00:00,13.534540
9,2022-02-17,36.661442,54.612771,1645110000000,9.245974,2022-02-17 15:00:00,13.773279
16,2022-02-18,36.349276,50.745742,1645196400000,9.129198,2022-02-18 15:00:00,12.744902
1,2022-02-19,36.995210,51.183249,1645282800000,9.110989,2022-02-19 15:00:00,12.605146
0,2022-02-20,35.435485,49.141487,1645369200000,9.065914,2022-02-20 15:00:00,12.572497
27,2022-02-21,37.993155,51.468442,1645455600000,9.009207,2022-02-21 15:00:00,12.204563
14,2022-02-22,38.287377,51.570473,1645542000000,8.929043,2022-02-22 15:00:00,12.026809
20,2022-02-23,43.832131,57.828715,1645628400000,8.422739,2022-02-23 15:00:00,11.112309
